In [1]:
import requests
import pandas as pd

# getting a list of IMDB ids

In [ ]:
# use links.csv to obtain imdb ids, then loop through some of them

# OMDB

In [20]:
#http://www.omdbapi.com/
# API-key d3de5220

In [ ]:
url = "http://www.omdbapi.com/?i=tt0076759&apikey=d3de5220"
result = requests.get(url)

In [ ]:
j = result.json()
df = pd.DataFrame(j)

In [16]:
def cleaning(df):
    df["Released"] = pd.to_datetime(df["Released"])
    def get_mins(x):
        y = x.replace(" min", "")
        return y
    df["Runtime"] = df["Runtime"].apply(get_mins) 
    df["Runtime"] = pd.to_numeric(df["Runtime"])
    return df

df = cleaning(df)

In [21]:
df.to_csv("data/OMDB.csv")